# Importing the dependanies

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# loading the data

In [11]:
# Load data
df = pd.read_csv(r"C:\Users\gasha\Desktop\solar-challenge-week0\data\benin-malanville.csv")
print("Dataset shape:", df.shape)

Dataset shape: (525600, 19)


# 1 Summary Statistics & Missing-Value Report

In [15]:
# Basic info
print("Dataset Info:",df.info())
#print(df.info())
#print("\nMissing Values:")
#print(df.isnull().sum())
#print(f"\nColumns with >5% nulls: {[col for col in df.columns if df[col].isnull().sum()/len(df) > 0.05]}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525600 entries, 0 to 525599
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Timestamp      525600 non-null  object 
 1   GHI            525600 non-null  float64
 2   DNI            525600 non-null  float64
 3   DHI            525600 non-null  float64
 4   ModA           525600 non-null  float64
 5   ModB           525600 non-null  float64
 6   Tamb           525600 non-null  float64
 7   RH             525600 non-null  float64
 8   WS             525600 non-null  float64
 9   WSgust         525600 non-null  float64
 10  WSstdev        525600 non-null  float64
 11  WD             525600 non-null  float64
 12  WDstdev        525600 non-null  float64
 13  BP             525600 non-null  int64  
 14  Cleaning       525600 non-null  int64  
 15  Precipitation  525600 non-null  float64
 16  TModA          525600 non-null  float64
 17  TModB          525600 non-nul